In [5]:
import bs4
from bs4 import BeautifulSoup
import requests
import time
import os

In [5]:
first_url = "https://www.findamasters.com/masters-degrees/msc-degrees/"
urls = "https://www.findamasters.com/masters-degrees/msc-degrees/?PG=2"

In [3]:
#get a list of the urls of all the 400 pages
def generate_urls(start_url, num_pages):
   url_list = [start_url.replace("?PG=2", f"?PG={i}") for i in range(1, num_pages+1)]
   return url_list

In [6]:
url_list = generate_urls(urls, 400)
url_list[0] = first_url

In [9]:
with open("masterdegree_url.txt", "w") as fl:
  for x in url_list:
      result_url = requests.get(x)
      result_soup = BeautifulSoup(result_url.text, 'html.parser')
      result_links = result_soup.find_all('a', {'class': 'courseLink'})
      for item in result_links:
        fl.write(f"www.findamasters.com{item.get('href')}\n")
      time.sleep(1)

In [2]:
with open('masterdegree_url.txt', 'r') as f:
   urls = [line.strip() for line in f]


In [11]:
r = 0
t = 15
for x in range(401):
    os.makedirs("pagina" + str(x))
    urls_15 = urls[r:t]
    num = 1
    for url in urls_15:
        response = requests.get("https://" + str(url))
        # Parse the HTML content
        soup = response.text
        #soup = BeautifulSoup(response.content, 'html.parser')
        # Save each link's HTML to a file in the corresponding directory
        # Save the HTML content to a file
        with open((f'pagina{x}/master{num}.html'), 'w') as f:
            f.write(soup)
        num += 1
    time.sleep(2)
    r = t 
    t = t +15   

### 7. Algorithm Question

### 7.1 Implement a code to solve the above mentioned problem.

In [ ]:
def subset_sum_with_ranges(nums, target_sum):
    '''dp[i][j] is True if there is a subset of the first i elements that adds up to j
       in this case we used dynamic programming to solve current problem
    '''
    n = len(nums)
    dp = [[False for _ in range(target_sum + 1)] for _ in range(n + 1)]

    # Base case: an empty subset can always add up to 0
    for i in range(n + 1):
        dp[i][0] = True

    # Fill the dp table using bottom-up dynamic programming
    for i in range(1, n + 1):
        for j in range(target_sum + 1):
            # Check if the subset sum can be formed without including the current range
            dp[i][j] = dp[i - 1][j]

            # Check if the subset sum can be formed by including the current range
            lowerbound, upperbound = nums[i - 1]
            for k in range(lowerbound, upperbound + 1):
                if j >= k:
                    dp[i][j] = dp[i][j] or dp[i - 1][j - k]

    # The final result is stored in the bottom-right cell of the dp table
    return dp[n][target_sum]

### 7.2 What is the time complexity (the Big O notation) of your solution? Please provide a detailed explanation of how you calculated the time complexity.

Time complexity "n=len(nums)"- O(1), function len() of array in python takes up O(1) time
Time complexity to calculate dp table creation is O(target_sum*len(nums)) because of 2 loops
Time complexity for base case: O(len(nums)), because of the for loop
Time complexity for the filling: O(len(nums) * target_sum * (max(nums)-min(nums))), where max(nums) is a max number among entire list of nums, min(nums) is a max number among entire list of nums
Time complexity of algorithm=O(len(nums) * target_sum * (max(nums)-min(nums)))+O(len(nums))+O(target_sum*len(nums))+O(1)=O(len(nums) * target_sum * (max(nums)-min(nums)))

### 7.3. Ask ChatGPT or any other LLM chatbot tool to check your code's time complexity (the Big O notation). Compare your answer to theirs. Do you believe this is correct? If the two differ, which one is right? (why?)

### Answer from chatgpt
The time complexity of the given code is O(n * target_sum * (max_range - min_range)), where:

n is the number of elements in the nums list.
target_sum is the target sum for which we are trying to find a subset.
max_range and min_range represent the upper and lower bounds for each range in the nums list.


Actually there is no difference between two results, because n is a number of elements in list, and max_range and min_range are the same as max(nums) and min(nums)

### 7.4. What do you think of the optimality of your code? Do you believe it is optimal? Can you improve? Please elaborate on your response.

Code is optimal because it is using dynamic programming, also considering the fact it is recursive problem, so from exponential cost we had been arrived to polinomial, which is good result 